First connect through SSH tunnel to tools-login.wmflabs.org server.

Replace the username and password (below) with those you find in the replica.my.cnf file in the host home

In [1]:
import pandas as pd
import numpy as np
import pymysql                                               # to get, do pip install PyMySQL
import json
from sklearn.feature_extraction.text import CountVectorizer  #from the NLP tutorial
import re                                                    #from NLP tutorial

In [3]:
#This connects to the Wiki database

username = 'u15068'                # get from replica.my.cnf
password = 'G5V2xrWbdFTSkyoU'      # get from replica.my.cnf
db=pymysql.connect(host='localhost',port=4711,user=username,passwd=password,db='enwiki_p')

In [5]:
#Get the epa id numbers from the dataset on github
# for edit protected articles: "070916_edit_protected_articles.csv"

nepa_articles = pd.read_csv("../Sampling/071816_batch1_sampled_ids.csv")   
print(nepa_articles.shape, len(nepa_articles))
nepa_ids=nepa_articles["page_id"]
nepa_ids[300:305]

(10026, 10) 10026


300      998108
301    24561971
302     6314225
303    49450359
304    11432297
Name: page_id, dtype: int64

In [6]:
# FUNCTION to Quary the wiki database for caterogylinks a page_id at a time
# To use it you need to be connected to a database through PyMySQL

# The result is a dictionary in the format {"page_id" : ["link1", "link2", ...]}

def GetDictWikiLinks(database, art_ids):
    data={}
    for i in range(len(art_ids)):       #to get them all len(epa_articles)
        cur = database.cursor()               #this is how you use pymusql apparently
        if i %1000 ==0:                 #progress print statement
            print("Quarry article", art_ids[i])
    
        #########################_____SQL_____##############################
        # This is the SQL command to quarry the Wikipedia database         #
        # The syntax of a quarry is:                                       #
        # SELECT [columns] FROM [table] WHERE [conditions] LIMIT [optional]#

# QUARRY FOR CATEGORYLINKS
#        quarry="""
#        SELECT cl_from, cl_to
#        FROM categorylinks
#        WHERE cl_from = %s"""

# QUARRY FOR PAGETITLES
#        quarry="""
#        SELECT page_id, page_title
#        FROM page

        quarry="""
        SELECT pl_from, pl_title
        FROM pagelinks
        WHERE pl_from = %s"""
        cur.execute(quarry, np.asscalar(art_ids[i]))
        links = np.array(cur.fetchall())
    
        #Some quarries might be empty 
        if links.size is 0:
            print("Empty categorylink %s", art_ids[i])
        else:   
            key=links[0,0].decode("utf-8")
            data[key]=[links[l,1].decode("utf-8") for l in range(len(links[:,1]))]
    
        #might take about 10 minuts (10k quarries, or 30k links)
    
    return data

In [7]:
# Get the dictionary from Wikipedia database

data=GetDictWikiLinks(db, nepa_ids)

Quarry article 7960121
Quarry article 940510
Quarry article 43629054
Quarry article 30602946
Quarry article 8663808
Quarry article 37880271
Quarry article 23566997
Quarry article 44340180
Quarry article 15397737
Quarry article 20617631
Quarry article 3064743


In [8]:
%matplotlib inline
len(data)

10026

In [9]:
df_nepa_links = pd.DataFrame.from_dict(data, orient="index")

In [10]:
df_nepa_links[:5]

,0,1,2,3,4,5,6,7,8,9,...,4055,4056,4057,4058,4059,4060,4061,4062,4063,4064
3893800,Abattoir_(band),Aggressive_Rock_Produktionen,And_One,Angry_Samoans,BMG_Rights_Management,Bathory_(band),Berlin,Billboard_(magazine),Bitch_(band),Black_Flag_(band),...,None,None,None,None,None,None,None,None,None,None
4468393,Alplaus_Maritime_Academy,Barn,Contributing_property,Cultural_artifact,Dutch_barn,Edmund_Andros,Erie_Canal,Geographic_coordinate_system,Historic_districts_in_the_United_States,History_of_the_National_Register_of_Historic_P...,...,None,None,None,None,None,None,None,None,None,None
71996,'Ndrangheta,2013_mass_surveillance_disclosures,ARM_architecture,AT&T,AT&T_Mobility,AT&T_Tilt,Advanced_Encryption_Standard,Android_(operating_system),Associated_Press,Atlanta,...,None,None,None,None,None,None,None,None,None,None
46416697,Cell_fusion,Cold_sores,Digital_object_identifier,Doctorate_of_Science,Gender_equality,Genital_herpes,Herpes_Simplex_Virus,Herpes_simplex_virus,Herpes_simplex_virus_1,Herpes_simplex_virus_2,...,None,None,None,None,None,None,None,None,None,None
4028353,Alex_Shane,All-England_Championship,All_Star_Wrestling,Andrew_Simmons,Atholl_Oakeley,Big_Brother_2007_(UK),British_National_Championship,Broxbourne,Chairman_(official),Channel_4,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
df_nepa_links.to_csv('nepa_links_titles_batch1_encoded2.csv', encoding='ascii')

In [ ]:
df_nepa_links.to_csv('nepa_links_titles_batch1.csv')

In [11]:
# Write and save data as a JSON to file

with open('pl_nepa_2.txt', 'w') as outfile:
    json.dump(data, outfile)

In [ ]:
        # Examples of querries!
        
        # to get the category links
        # SELECT cl_from, cl_to
        # FROM categorylinks
        # WHERE cl_from = %s
        
        # To get the page-links
        # SELECT pl_from, pl_title
        # FROM pagelinks
        # WHERE pl_from = %s
        # AND pl_namespace = 0
        
        # to get stuff from different tables:
        # SELECT page_id, cl_to
        # FROM page inner join categorylinks on(page.page_id = categorylinks.cl_from)
        # WHERE page_id = %s
        # actually that could have been simplyfied, but it nicely shows how to merge tables with "inner join"

########################################################

Now I need to transform the categorylinks file by splitting the links in individual words

I will define a function called GetWordsList()

    cl_epa.txt 
    cl_words_epa.txt

In [ ]:
import json

In [ ]:
with open("cl_epa.txt", "r") as f:
    data=json.load(f)
len(data)

In [ ]:
data["25"][:3]

In [ ]:
# This function is to get the same files but with individual words separated 

import re

def GetWordList(data_dict):
    #data_dict is a dictionary {keys}:["str1", "str2",..., "strN"]
    #this functions yields ["str1", "str2", ... "strM"], where M is the sum of N over all Keys
    list_categories = []
    cl_index=[]
    for key in data_dict: 
        cl_index.append(key)
        links=[]
        for n in range(len(data_dict[key])):
        
            # Remove the following regular expressions
            
            re0="protected_page"
            #re1="with_accessdate"
            #re2=".rticle"
            re4=".ikipedia"
            #re4="to_be_expanded"
            #re5="with_unsourced_statements"
            #re6="needing_additional_references"
            #re7="lacking_sources"
            #re8="containing_potentially_dated_statements"
            #re9="with_dead_external_links"
            re1="Certification_Table_Entry_usages_for"
            re2="language_sources"
            re3="Articles_containing"
            
            if not any(re.search(regex, data_dict[key][n]) for regex in [re0, re1, re2, re3, re4]):
                words=data_dict[key][n].replace("_", " ")
                links.append(words)
        links=" ".join(links)
        list_categories.append(links)
                
    return list_categories, cl_index      

In [ ]:
clean_categories, epa_id = GetWordList(data)

In [ ]:
len(clean_categories)
clean_categories[1]

In [ ]:
epa_id[1]

In [ ]:
data[epa_id[1]][:5]

In [ ]:
words_dict={}

for i in range(len(clean_categories)):
    words_dict[epa_id[i]]=clean_categories[i]

In [ ]:
words_dict["19283178"]

In [ ]:
with open('cl_words_epa.txt', 'w') as outfile:
    json.dump(words_dict, outfile)

In [ ]:
with open("cl_words_epa.txt", "r") as f:
    clw=json.load(f)
len(clw)

In [ ]:
clw["13259888"]

In [ ]:
for i in range(len(clean_categories)):
    
link=clean_categories[0].split()
words=link[1].split("_")
print(words)

In [ ]:
#then I would do the same thing, but parsing the words from the categories separately
#Then the same thing but including all the links in the article from the other table

You can keep the really specific topics that occur above some threshold of pages (5%? 10%?), then split the topics into words, filter stop words, and the bag of words to get things that are common enough to model.

In [ ]:
#I should try and run my own random forest, but first
#I should get a sample of non edit-protected articles (what if they have other kinds of protection?)
#then mix them with my dataset, make sure they are labelled, split in training and validation sets 
#train a random forest algorithm
#To do that I will have to modify the query and I might want to branch off this notebook, so first I should push it on github 
#And this is a good time to re-practice that